# **Launch Sites Locations Analysis with Folium**

## Objectives


*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

After completed the above Objectives, we should be able to find some geographical patterns about launch sites.


In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [2]:
##  Mark all launch sites on a map
world_map = folium.Map()
spacex_df=pd.read_csv("dataset_part_2.csv")

Now, we can take a look at what are the coordinates for each site.

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['LaunchSite','Latitude' , 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608059,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Now, let's add a circle for each launch site in data frame `launch_sites`



In [38]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Latitude'], row['Longitude']]
    folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['LaunchSite'])).add_to(site_map)
    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'], )).add_to(site_map)
site_map


now,we have marked our launch sites.we can see that all the launch sites are near the coast line in other words all the launch sites are near the ocean

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [7]:
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class
80,CCSFS SLC 40,28.561857,-80.577366,1
81,CCSFS SLC 40,28.561857,-80.577366,1
82,CCSFS SLC 40,28.561857,-80.577366,1
83,CCSFS SLC 40,28.561857,-80.577366,1
84,CCSFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608059,-80.603956,1
86,KSC LC 39A,28.608059,-80.603956,1
87,KSC LC 39A,28.608059,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1
89,CCSFS SLC 40,28.561857,-80.577366,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

first we  Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [8]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class,marker_color
80,CCSFS SLC 40,28.561857,-80.577366,1,green
81,CCSFS SLC 40,28.561857,-80.577366,1,green
82,CCSFS SLC 40,28.561857,-80.577366,1,green
83,CCSFS SLC 40,28.561857,-80.577366,1,green
84,CCSFS SLC 40,28.561857,-80.577366,1,green
85,KSC LC 39A,28.608059,-80.603956,1,green
86,KSC LC 39A,28.608059,-80.603956,1,green
87,KSC LC 39A,28.608059,-80.603956,1,green
88,CCSFS SLC 40,28.561857,-80.577366,1,green
89,CCSFS SLC 40,28.561857,-80.577366,1,green


Let's first create a `MarkerCluster` object
then For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [9]:
# Add marker_cluster to current site_map
marker_cluster = MarkerCluster()

site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, row in spacex_df.iterrows():
    # create and add a Marker cluster to the site map
    coordinate = [row['Latitude'], row['Longitude']]
    folium.map.Marker(coordinate, icon=folium.Icon(color='white',icon_color=row['marker_color'])).add_to(marker_cluster)
site_map

Next, we need to explore and analyze the proximities of launch sites.
Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [10]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

First we have to Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site, and we can Draw a `PolyLine` between a launch site to the selected coastline point

In [37]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)


def coast(a,b,c,d,e):
    distance_coastline = calculate_distance(a, b, c, d)
    print(e,distance_coastline,'km ')
    distance_marker = folium.Marker(  
       [c, d],
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
           )
       )
    coordinates = [[a,b],[c,d]]
    lines=folium.PolyLine(coordinates, color = 'red',weight = 1).add_to(site_map)
    return coast
kslc = coast(28.60807,-80.604,28.61109,-80.59673,'kslc')
ccsfc = coast(28.56181,-80.57723,28.56315,-80.56779,'ccfc')
vafb = coast(34.63222,-120.61143,34.63434,-120.62568,'vafb')
site_map.add_child(distance_marker)
site_map.add_child(lines)
site_map

kslc 0.7853712068015124 km 
ccfc 0.9341863572101344 km 
vafb 1.325315568069371 km 
